# モデルのロギングとデプロイ

## デプロイの流れ

1. **LoggedModelとしてロギング**: RAGパイプラインを`mlflow.pyfunc.PythonModel`としてラップ
2. **Unity Catalogに登録**: `mlflow.register_model()`でUnity Catalogに登録
3. **サービングエンドポイントにデプロイ**: `databricks.agents.deploy()`でデプロイ
4. **エンドポイントのテスト**: デプロイされたエンドポイントを呼び出してテスト

内容
- LoggedModelの実装方法
- Unity Catalogへの登録
- agents.deployを使ったデプロイ
- 本番環境でのモニタリング設定
- Scorerの登録とアクティブ化
- サンプリング設定とバックフィル

参考リンク：
- [MLflow3とDatabricksで実現するLLMops](https://qiita.com/taka_yayoi/items/2fd4c9fef0ffe8377f48)
- [Agent評価とデプロイ（日本語）](https://docs.databricks.com/ja/generative-ai/deploy-agent.html)
- [本番環境モニタリング（日本語）](https://docs.databricks.com/aws/ja/mlflow3/genai/eval-monitor/production-monitoring)


## LoggedModelとしてロギング

### 🎯 このステップの目的

アプリケーションロジックをMLflow 3の`ResponsesAgent`インタフェースに準拠した形式で実装し、
**Logged Model**として記録することで、プロンプト・トレース・モデルを統合的に管理します。

### 📚 背景知識

**Logged Modelとは？（最重要コンセプト）**

Logged ModelはMLflowにおける**一級エンティティ（First-class Entity）**で、
生成AIアプリケーションのライフサイクル全体を管理する中心的な概念です。

**Logged Modelの重要性**
1. **統一されたモデル表現**: 従来のMLモデルから複雑なGenAIエージェントまで、単一の抽象化で管理
2. **観測性の統合**: トレース、評価結果、メトリクスが自動的にリンクされ、モデルの挙動を包括的に可視化
3. **プロンプトとの相互リンク**: プロンプトレジストリと双方向リンクを構築（後述）
4. **ライフサイクル管理の中心**: 開発→評価→デプロイ→監視の全フェーズでモデルを軸に管理

参考:
- [Logged Model概念（英語）](https://mlflow.org/docs/3.2.0rc0/genai/data-model/logged-model/)
- [log_model API（英語）](https://mlflow.org/docs/latest/python_api/mlflow.pyfunc.html#mlflow.pyfunc.log_model)

**ResponsesAgentとは？**
- MLflow 3で導入された生成AIアプリケーション用の標準インタフェース
- OpenAI Responses形式との互換性を提供
- トレーシング、評価、デプロイを統合的にサポート
- 参考: [ResponsesAgent API（英語）](https://mlflow.org/docs/latest/python_api/mlflow.pyfunc.html#mlflow.pyfunc.ResponsesAgent)

**ResponsesAgentの要件**
1. `mlflow.pyfunc.ResponsesAgent`を継承
2. `predict()`メソッドで`ResponsesAgentRequest`を受け取る
3. `ResponsesAgentResponse`を返す
4. 入力スキーマは`{"input": [{"role": ..., "content": ...}]}`形式

### 💡 実装のポイント

**1. Logged Modelの作成: models from codeアプローチ**

`mlflow.pyfunc.log_model()`を使用してLogged Modelを作成します：
- モデルクラスをPythonファイル（`customer_support_agent.py`）として定義
- `python_model="customer_support_agent.py"`でファイルパスを指定
- デプロイ時にコードが自動的にパッケージ化される
- MLflow Runに関連付けられ、バージョン管理・再現性を確保

**2. Logged Modelとプロンプトの相互リンク（重要！）**

`prompts`パラメータを使用して**双方向リンク**を構築します：

```python
mlflow.pyfunc.log_model(
    name="agent_model",
    python_model="customer_support_agent.py",
    prompts=[prompt_uri]  # ← ここで相互リンクを構築
)
```

**相互リンクの効果**:
- Logged Model → Prompt: モデルのメタデータ（MLmodelファイル）に関連プロンプトが記録
- Prompt → Logged Model: Prompt Registry UIからモデルへ移動可能
- プロンプト変更時の影響範囲を追跡しやすくなる
- `model_info.prompts`でプログラム的にアクセス可能

**補完的な記録方法**（3重の記録でさらに堅牢に）:
1. ✅ `prompts=[...]`パラメータ（**MLflow 3推奨・最重要**）
2. MLflowタグ（`prompt_name`, `prompt_alias`, `prompt_uri`）← 検索・フィルタリング用
3. `model_config`（設定値として埋め込み）← モデルロード時の動的プロンプトロード用

**3. 注意点**
- OutputItemは`.content`を使用（`.text`ではない）
- `mlflow.openai.autolog()`でLLM呼び出しを自動トレース
- `@mlflow.trace`デコレータでカスタムスパンを追加

In [0]:
# MLflowとDatabricks SDKのインストール
%pip install mlflow databricks-sdk databricks-agents databricks-langchain
dbutils.library.restartPython()

In [0]:
%run ./00_config

In [0]:
import mlflow
from mlflow.models import infer_signature

# Prepare example input and output
input_example = {"query": "福永敦史の研究概要は？"}
# output_example = "福永敦史さんの研究は、行動認識技術を使って、大学構内における不法投棄を自動で検知するシステムの開発です。"

# Infer signature
# signature = infer_signature(input_example, output_example)

with mlflow.start_run() as run:
    logged_model_info = mlflow.pyfunc.log_model(
        artifact_path="rag_pipeline_model",
        python_model=f"{src_directory_path}/rag_pipeline.py",
        pip_requirements=[
            "mlflow>=3.1.0",
            "databricks-sdk",
            "databricks-vector-search"
        ],
        # signature=signature,
        input_example=input_example
    )

    # モデル情報を保存
    model_uri = logged_model_info.model_uri
    run_id = run.info.run_id

    print(f"✅ Logged Model作成完了")
    print(f"   Model URI: {model_uri}")
    print(f"   Run ID: {run_id}")
    # print(f"   Prompt: {prompt_uri}")

In [0]:
loaded_model = mlflow.pyfunc.load_model(model_uri)

# ダミーのテストデータ（評価データセットはまだ定義されていないため）
test_query = {
    "query": "福永敦史の研究の概要を教えて"
}

print("\n📊 ResponsesAgent LoggedModelのテスト実行:")
# 辞書形式で直接予測
result = loaded_model.predict(test_query)

print("\n✅ ResponsesAgent LoggedModelの作成とテスト完了（UC登録前）")

## 評価

In [0]:
import pandas as pd

# 評価用のQAデータセット
# mlflow.genai.evaluate()では 'inputs' と 'expectations' カラムが必要
# inputs と expectations はどちらも辞書形式である必要がある
eval_data = pd.DataFrame({
    "inputs": [
        {"question": "福永敦史の研究概要は？"},
        {"question": "福永敦史の研究結果は？"},
    ],
    "expectations": [
        {"expected_response": "福永敦史さんの研究は、行動認識技術を使って、琉球大学の構内で不法投棄を検知するシステムを開発したものです。"},
        {"expected_response": "「不法投棄」クラスの再現率が課題となっており、データ拡張とモデル高速化が今後の重要な改善点です。"},
    ]
})

print("=== 評価データセット ===")
display(eval_data)

In [0]:
import mlflow

loaded_model = mlflow.pyfunc.load_model(model_uri)

# 予測を生成（テスト用）
eval_data["prediction"] = eval_data["inputs"].apply(
    lambda x: str(loaded_model.predict({"query": x["question"]}))
)

print("=== 予測結果 ===")
display(eval_data[["inputs", "prediction"]])

### 事前構築されたJudgeを使った評価

MLflowの事前構築されたJudge（評価者）を使います。

In [0]:
from mlflow.genai.scorers import RelevanceToQuery, Correctness, Safety, RetrievalGroundedness, Guidelines

# 評価用のpredict関数を作成
# predict_fnの引数名は、inputsカラムの辞書のキーと一致する必要がある
def predict_fn(question):
    """
    評価用のpredict関数
    question: 質問文（文字列）
    returns: 予測結果（文字列）
    """
    print(question)

    result = loaded_model.predict({"query": question})
    return str(result)

# Guidelines Judge（ガイドライン準拠）の作成
guidelines = Guidelines(
    guidelines="""
    良い回答の基準:
    1. 簡潔で分かりやすい（50-300文字）
    2. 専門用語を適切に使用
    3. 具体的な説明を含む
    """
)

# 評価の実行
with mlflow.start_run(run_name="QA Model Evaluation with All Judges"):

    # すべての事前構築されたJudge（評価者）を定義
    judges = [
        RelevanceToQuery(),        # 質問への関連性
        Correctness(),             # 正確性（グラウンドトゥルースと比較）
        Safety(),                  # 安全性（有害コンテンツの検出）
        RetrievalGroundedness(),   # 幻覚の検出
        guidelines                 # ガイドライン準拠
    ]

    print("=== 使用するJudges ===")
    print("1. RelevanceToQuery: 質問への関連性")
    print("2. Correctness: 正確性（グラウンドトゥルースとの比較）")
    print("3. Safety: 安全性（有害コンテンツの検出）")
    print("4. RetrievalGroundedness: 幻覚の検出")
    print("5. Guidelines: カスタムガイドライン準拠")
    print()

    # mlflow.genai.evaluateで評価
    eval_results = mlflow.genai.evaluate(
        data=eval_data,
        predict_fn=predict_fn,
        scorers=judges
    )

    print("=== 評価結果 ===")
    print(f"\n評価スコア:")
    for metric_name, metric_value in eval_results.metrics.items():
        print(f"  {metric_name}: {metric_value:.3f}")

    print("\n✅ すべてのJudgeによる評価が完了しました！")
    print("📊 詳細な評価結果は右側の「Experiments」タブから確認できます")

## Unity Catalogにモデルを登録

In [0]:
APP_NAME = "mydocs_assistant"

# TODO
version_identifier = "1"

# Unity Catalogモデル名の定義（version_identifierはステップ3で定義済み）
agent_model_name = f"{APP_NAME}_{version_identifier}"  # UC命名規則に準拠
UC_MODEL_NAME = f"{catalog}.{schema}.{agent_model_name}"  # Unity Catalog 3レベル名前空間

print(f"🗂️ Unity Catalogにモデルを登録中: {UC_MODEL_NAME}")
print(f"   モデルURI: {model_uri}")

try:
    # モデルをUnity Catalogに登録
    registered_model = mlflow.register_model(
        model_uri=model_uri,
        name=UC_MODEL_NAME,
        tags={
            "mlflow_trace_enabled": "true",
            "model_framework": "agent_framework",
            "model_version": version_identifier
        }
    )
    
    UC_MODEL_VERSION = registered_model.version
    
    print(f"✅ UCモデル登録完了:")
    print(f"   名前: {UC_MODEL_NAME}")
    print(f"   バージョン: {UC_MODEL_VERSION}")
    print(f"   アプリケーションロジック: 含まれています")
    
    # オプション: モデルの説明を追加
    from mlflow import MlflowClient
    client_mlflow = MlflowClient()
    
    client_mlflow.update_model_version(
        name=UC_MODEL_NAME,
        version=UC_MODEL_VERSION,
        description=f"Agent Framework互換のカスタマーサポートアプリケーション (Version: {version_identifier})"
    )
    
    # モデルバージョンにタグを追加
    client_mlflow.set_model_version_tag(
        name=UC_MODEL_NAME,
        version=UC_MODEL_VERSION,
        key="mlflow_trace_model_id",
        value=model_uri
    )
    
except Exception as e:
    print(f"⚠️ モデル登録エラー: {e}")
    # エラーの場合でも継続できるよう、デフォルト値を設定
    UC_MODEL_VERSION = 1
    print(f"   デフォルトバージョン使用: {UC_MODEL_VERSION}")

## 本番監視用スコアラーの設定（デプロイ前に実行）

### 🎯 このステップの目的

**重要**: エンドポイントをデプロイする**前に**スコアラーを登録・開始します。

### ⚠️ なぜデプロイ前にスコアラーを設定するのか？

**検証結果により判明した重要な仕様**:

| 順序 | Trace Metrics Computation Jobの状態 | 結果 |
|------|-----------------------------------|------|
| **デプロイ前にスコアラー作成** | **Active** ✅ | 自動評価が正常に動作 |
| デプロイ後にスコアラー作成 | Paused ❌ | 手動Resumeが必要 |

**システムの動作**:
- スコアラーを先に登録・開始すると、Trace Metrics Computation Jobが**Active状態**で作成される
- エンドポイントデプロイ後、トレースが記録されると自動的に評価が開始される
- デプロイ後にスコアラーを作成すると、ジョブがPaused状態で作成され、手動Resumeが必要になる

**参考**: この動作はDatabricksドキュメントには明記されていませんが、実際の検証で確認された仕様です。

### 📚 本番監視（Production Monitoring）とは？

- 本番トラフィックに対してスコアラーを自動実行する機能
- バックグラウンドジョブとして15-20分ごとにバッチ処理される
- 参考: [Production monitoring](https://docs.databricks.com/ja/mlflow3/genai/eval-monitor/production-monitoring.html)

### 💡 実装のポイント

**サンプリング率**
- `sample_rate=1.0`: 100%のトレースを評価
- すべてのトレースを評価することで品質を完全に監視

**評価のタイミング**
- スコアラー開始時にTrace Metrics Computation JobがActive状態で作成される
- エンドポイントデプロイ後、トレースが記録されると自動評価が開始される
- 初回評価まで15-20分かかる

In [0]:
from mlflow.genai.scorers import Safety, Guidelines, ScorerSamplingConfig

EXPERIMENT_NAME = "/Users/a2c6201@gmail.com/databricks-rag/notebooks/04_logging_model"
# エクスペリメントIDを取得
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
experiment_id = experiment.experiment_id

print(f"📊 本番監視スコアラーを設定中（デプロイ前）...")
print(f"   Experiment: {EXPERIMENT_NAME}")
print(f"   Experiment ID: {experiment_id}")

# Safety スコアラー（オフライン評価と同じデフォルト名を使用）
safety_scorer = Safety()
safety_scorer = safety_scorer.register(experiment_id=experiment_id)
safety_scorer = safety_scorer.start(
    experiment_id=experiment_id,
    sampling_config=ScorerSamplingConfig(sample_rate=1.0)
)

# Guidelines スコアラー（オフライン評価と同じ名前を使用）
guidelines_scorer = Guidelines(
    name="guidelines_scorer",
    guidelines=[
        "応答は丁寧で親切であること",
        "応答は50-500文字程度であること",
        "会社名TechCorpに言及すること"
    ]
)
guidelines_scorer = guidelines_scorer.register(experiment_id=experiment_id)
guidelines_scorer = guidelines_scorer.start(
    experiment_id=experiment_id,
    sampling_config=ScorerSamplingConfig(sample_rate=1.0)
)

print(f"\n✅ 本番監視スコアラー登録・開始完了")
print(f"   - Safety (デフォルト名)")
print(f"   - guidelines_scorer (Guidelines)")
print(f"   - サンプリング率: 100%")
print(f"\n💡 Trace Metrics Computation JobがActive状態で作成されました")
print(f"   Workflows > Jobs で 'Trace Metrics Computation' を検索して確認できます")

## デプロイ

Unity Catalogに登録したモデルをサービングエンドポイントにデプロイします。
トレースが正しく動作するよう環境変数を適切に設定します。

**注**: スコアラーは前ステップで設定済みなので、デプロイ後に自動的に評価が開始されます。


![mydocs_assistant_agent_1_-_Databricks.png](./mydocs_assistant_agent_1_-_Databricks.png "mydocs_assistant_agent_1_-_Databricks.png")

![Cursor_と_サービングエンドポイントを作成する.png](./Cursor_と_サービングエンドポイントを作成する.png "Cursor_と_サービングエンドポイントを作成する.png")